In [ ]:
import pandas as pd
import os
import uuid
from datetime import datetime
from openpyxl import load_workbook

# Define file paths
db_file = r"C:\your path\banking_system.xlsx"
authorized_bankers_file = r"C:\your path\authorized_bankers.xlsx"

class BankingSystem:
    def __init__(self):
        self.accounts_columns = ["Account Number", "Account Holder", "Password", "Role"]
        self.transactions_columns = ["Transaction ID", "Account Number", "Transaction Date", "Transaction Type", "Amount", "Previous Balance", "New Balance"]
        self.initialize_db()
    
    def initialize_db(self):
        if not os.path.exists(db_file):
            with pd.ExcelWriter(db_file, engine='openpyxl') as writer:
                pd.DataFrame(columns=self.accounts_columns).to_excel(writer, sheet_name="Accounts", index=False)
                pd.DataFrame(columns=self.transactions_columns).to_excel(writer, sheet_name="Transactions", index=False)
    
    def load_data(self, sheet_name):
        return pd.read_excel(db_file, sheet_name=sheet_name, dtype=str)
    
    def save_data(self, df, sheet_name):
        with pd.ExcelWriter(db_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)

class User:
    def __init__(self, banking_system):
        self.banking_system = banking_system

    def register(self, role):
        acc_holder = input("Enter Account Holder Name: ").strip()
        password = input("Enter Password: ").strip()
        
        if role == "Admin":
            authorized_df = pd.read_excel(authorized_bankers_file, dtype=str)
            if acc_holder not in authorized_df['Account Holder'].astype(str).values:
                print("Unauthorized admin registration.")
                return
            acc_number = input("Enter Admin ID as Account Number: ").strip()
        else:
            acc_number = str(uuid.uuid4().int)[:10]  # Auto-generate account number
        
        df = self.banking_system.load_data("Accounts")
        new_user = pd.DataFrame([[acc_number, acc_holder, password, role]], columns=self.banking_system.accounts_columns)
        df = pd.concat([df, new_user], ignore_index=True)
        self.banking_system.save_data(df, "Accounts")
        print(f"Registration successful! Your account number is: {acc_number}")
    
    def login(self):
        acc_number = input("Enter Account Number: ").strip()
        password = input("Enter Password: ").strip()
        df = self.banking_system.load_data("Accounts")
        df['Account Number'] = df['Account Number'].astype(str)
        df['Password'] = df['Password'].astype(str)
        user = df[(df['Account Number'] == acc_number) & (df['Password'] == password)]
        
        if user.empty:
            print("Invalid login credentials!")
            return None
        
        print("Login successful!")
        return user.iloc[0]

class Transactions:
    def __init__(self, banking_system):
        self.banking_system = banking_system

    def view_accounts(self):
        df = self.banking_system.load_data("Accounts")[["Account Number", "Account Holder", "Role"]]
        print(df)
    
    def view_transactions(self, account_number):
        df = self.banking_system.load_data("Transactions")
        df = df[df['Account Number'].astype(str) == account_number]
        print(df)
    
    def perform_transaction(self, account_number, transaction_type):
        df_acc = self.banking_system.load_data("Accounts")
        df_trans = self.banking_system.load_data("Transactions")
        
        acc_row = df_acc[df_acc['Account Number'].astype(str) == account_number]
        if acc_row.empty:
            print("Account not found!")
            return
               
        amount = float(input("Enter Amount: "))
        df_trans['New Balance'] = pd.to_numeric(df_trans['New Balance'], errors='coerce').fillna(0)
        
        # Calculate previous balance
        previous_balance = df_trans[df_trans['Account Number'].astype(str) == account_number]['New Balance'].iloc[-1]
        previous_balance = 0 if pd.isna(previous_balance) else previous_balance
                        
        new_balance = previous_balance + amount if transaction_type == "Deposit" else previous_balance - amount
        
        transaction_id = str(uuid.uuid4().int)[:10]  # Auto-generate transaction ID
        new_transaction = pd.DataFrame([[transaction_id, account_number, datetime.now().strftime("%Y-%m-%d %H:%M:%S"), transaction_type, amount, previous_balance, new_balance]], columns=self.banking_system.transactions_columns)
        df_trans = pd.concat([df_trans, new_transaction], ignore_index=True)
        self.banking_system.save_data(df_trans, "Transactions")
        print(f"{transaction_type} successful! New balance: {new_balance}")

class BankingApp:
    def __init__(self):
        self.banking_system = BankingSystem()
        self.user_system = User(self.banking_system)
        self.transaction_system = Transactions(self.banking_system)
    
    def main_menu(self):
        while True:
            choice = input("Choose option: 1. Register User 2. Register Admin 3. Login 4. Exit: ")
            if choice == '1':
                self.user_system.register("User")
            elif choice == '2':
                self.user_system.register("Admin")
            elif choice == '3':
                user = self.user_system.login()
                if user is not None:
                    while True:
                        if user['Role'] == "Admin":
                            admin_choice = input("1. View All Accounts 2. View Transactions 3. Logout: ")
                            if admin_choice == '1':
                                self.transaction_system.view_accounts()
                            elif admin_choice == '2':
                                acc_number = input("Enter Account Number to View Transactions: ")
                                self.transaction_system.view_transactions(acc_number)
                            elif admin_choice == '3':
                                break
                        else:
                            user_choice = input("1. View Account Details 2. View Transactions 3. Deposit 4. Withdraw 5. Logout: ")
                            if user_choice == '1':
                                print(user)
                            elif user_choice == '2':
                                self.transaction_system.view_transactions(user['Account Number'])
                            elif user_choice == '3':
                                self.transaction_system.perform_transaction(user['Account Number'], "Deposit")
                            elif user_choice == '4':
                                self.transaction_system.perform_transaction(user['Account Number'], "Withdraw")
                            elif user_choice == '5':
                                break
            elif choice == '4':
                print("Exiting application...")
                break

if __name__ == "__main__":
    app = BankingApp()
    app.main_menu()
